In [ ]:
!pip install flwr[simulation] torch torchvision
!pip install fedartml
!pip install keras
!pip install --upgrade flwr

import flwr as fl
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, TensorDataset
from fedartml import InteractivePlots
from keras.datasets import mnist
from torchvision import datasets, transforms
from fedartml import SplitAsFederatedData
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from flwr.server import ServerConfig
import random
from flwr.common import Context, log, parameters_to_ndarrays, ndarrays_to_parameters,  FitIns
from sklearn.cluster import KMeans
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from flwr.server.strategy import FedAvg
from sklearn.preprocessing import normalize
import os

#GLOBAL PARAMETERS

#Experiment ID and mode
EXPERIMENT = 2
"""MODES are: fedavg for normal fedavg that takes all clients,
              randselFedavg is fedAvg with lower fraction fit to randomly select clients at every round,
              WLRC experiments with client selection based on the estimated rademacher complexity,
              LRC_Sampling is for experiment 2, it changes the client selection strategy to selecting the lowest r-score client out of every cluster
              Mixed_sampling tries to both integrate the "best" cluster and also to integrate  every best node from every cluster. Again, useful for experiment 2 only."""
MODE = 'WLRC' #fedavg or WLRC or randselFedavg or LRC_Sampling for extremely high nonIIDness (experiment 2)
PERMITTED_MODES = ('fedavg', 'randselFedavg', 'WLRC', 'LRC_Sampling', 'Mixed_Sampling')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#Hyperparameters of FL setting
SEED = 18
NUM_ROUNDS = 20 #Num strategy rounds for the simulation
WLRC_ROUNDS = {10} #Rounds at which we activate clustering and the selection
NUM_CLIENTS = 100 #NumClients of the simulation
ALPHA = 0.1 #alpha parameter of FedArtML
T = 10 #Num of local training epochs on each client
PREFIX_PATH = "/content/drive/MyDrive/FL_WLRC/"
NUM_CLUSTERS = 10
lr = 0.001 #Learning rate
fraction_fit = 0.1 if MODE == 'randselFedavg' else 1 #fraction of clients used for training in FedAvg step

#Rademacher complexity parameters
Q = 100 #Number of times we sample the variables for rademacher complexity
C = 10 #Number of classes
TOP_K = 1

#Open drive
from google.colab import drive
drive.mount('/content/drive')

os.makedirs(os.path.join(PREFIX_PATH, f"Experiment_{EXPERIMENT}"), exist_ok=True)
LOG_DIR = os.path.join(PREFIX_PATH, f"Experiment_{EXPERIMENT}/{MODE}/" + ("high_skew/" if ALPHA == 0.5 else "") + ("more_rounds/" if NUM_ROUNDS > 10 and 10 not in WLRC_ROUNDS else "") +
 ("delayed_clustering/" if NUM_ROUNDS > 10 and (10 in WLRC_ROUNDS and 5 not in WLRC_ROUNDS) else ""))
os.makedirs(LOG_DIR, exist_ok=True)

#SEEDING
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
generator = torch.Generator()
generator.manual_seed(SEED)

Mounted at /content/drive


# To Run

## Model

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


## Dataset

In [ ]:
(x_train_glob, y_train_glob), (x_test_glob, y_test_glob) = mnist.load_data()
y_train_glob = np.reshape(y_train_glob, (y_train_glob.shape[0],))
y_test_glob = np.reshape(y_test_glob, (y_test_glob.shape[0],))

# Normalize pixel values to be between 0 and 1
x_train_glob, x_test_glob = x_train_glob / 255.0, x_test_glob / 255.0

# Instantiate a SplitAsFederatedData object
my_federater = SplitAsFederatedData(random_state = SEED)

# Get federated dataset from centralized dataset
clients_glob_dic, list_ids_sampled_dic, miss_class_per_node, distances = my_federater.create_clients(image_list = x_train_glob, label_list = y_train_glob,
                                                             num_clients = NUM_CLIENTS, prefix_cli='Local_node', method="dirichlet", alpha=ALPHA)

#my_plot = InteractivePlots(labels = y_train_glob)
#my_plot.show_bar_divided_distr_dirichlet()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def is_iid(node, clients_glob_dic):
  labels = set()
  for i in clients_glob_dic["without_class_completion"][f"Local_node_{node}"]:
    labels.add(i[1])
    if len(labels) == C:
      return True
  return False

def get_distribution(node, clients_glob_dic):
  distr = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 0: 0}
  for i in clients_glob_dic["without_class_completion"][f"Local_node_{node}"]:
    distr[i[1]] += 1
  return distr

In [ ]:
cnt = 0
iid_clients = []
for i in range(1,NUM_CLIENTS+1):
  #if clients_glob_dic["without_class_completion"][f"Local_node_{i}"] == clients_glob_dic["with_class_completion"][f"Local_node_{i}"]:
  if is_iid(i, clients_glob_dic):
    cnt += 1
    d = get_distribution(i, clients_glob_dic)
    print(f"Node {i} iid with: {d}")
    iid_clients.append(i)
print(cnt)



Node 100 iid with: {1: 29, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 4, 9: 1, 0: 1}
1


## Rademacher Complexity Estimation & Aggregation

In [ ]:
def sample_rademacher_variables(batch_size, device, num_classes=10):
    return torch.randint(0, 2, (batch_size, num_classes), device=device) * 2 - 1 #get random variables in {-1, 1}

def estimate_Rk(Q, R, B, C, model, batch_x):
  device = batch_x.device
  Rk_est = 0

  for i in range(Q):
    #Get rademacher variables and model output
    r_variables = sample_rademacher_variables(B, device, C)
    out = model(batch_x)

    #multiply them  and sum
    z = torch.sum(out * r_variables)

    #Update rademacher estiamtion
    Rk_est += torch.abs(z / (B * C))

  return Rk_est / Q


def aggregate_and_rank_score_per_cluster(r_per_node, cluster_assignments):
    """  Cluster assignment is a dictionary of node ids with their cluster,
        r_per_node is a dictionary of nodes with their rademacher complexity
        returns a dictionary of clusters with their average rademacher complexity and an ordered list of cluster labels, from least to highest score
        Normal WLRC uses this function to select the clusters to train on
    """
    r_per_cluster = {}
    for node, r in r_per_node.items():
        cluster = cluster_assignments[node]
        if cluster not in r_per_cluster:
            r_per_cluster[cluster] = []
        r_per_cluster[cluster].append(r)

    #aggregate through averaging
    for cluster, r_list in r_per_cluster.items():
        r_per_cluster[cluster] = sum(r_list) / len(r_list)

    #order by score
    ranking = [cluster for cluster, _ in sorted(r_per_cluster.items(), key=lambda x: x[1])]

    return r_per_cluster, ranking

#Get, for every cluster, top_k nodes in rademacher score to consider for training (default k is 1), used in LRC_Sampling mode
def get_top_k_nodes_per_cluster(k, r_per_node, cluster_assignments):
    """  Cluster assignment is a dictionary of node ids with their cluster,
        r_per_node is a dictionary of nodes with their rademacher complexity
        k is the number of nodes to consider per cluster
        returns a dictionary of clusters with their top k rademacher complexity nodes
        LRC_Sampling mode uses this function to select the nodes to train on
        returns set of nodes chosen for training
    """
    cluster_scores_and_assignments = {}
    for node, r in r_per_node.items():
        cluster = cluster_assignments[node]
        cluster_scores_and_assignments[cluster] = cluster_scores_and_assignments.get(cluster, []) + [(r, node)]

    top_k_nodes_per_cluster = {}
    for cluster in cluster_scores_and_assignments:
        cluster_scores_and_assignments[cluster].sort(key=lambda x: x[0])
        top_k_nodes_per_cluster[cluster] = [node for _, node in cluster_scores_and_assignments[cluster][:k]]

    #return list of chosen nodes
    node_choice = [node for nodes in top_k_nodes_per_cluster.values() for node in nodes]
    return set(node_choice)



## Strategy

In [ ]:
flowerids2cid = {}
class CustomFedAvg(FedAvg):
    def __init__(self, *args, **kwargs):
        self.round = 0
        self.client_clusters = {}  # client id -> cluster id
        self.num_clusters = kwargs.pop("num_cluster")
        self.mode = kwargs.pop("mode")
        assert self.mode in PERMITTED_MODES, "Mode must be in PERMITTED MODES"
        super().__init__(*args, **kwargs)

    def aggregate_fit(self, server_round, results, failures):
        self.round = server_round  # update current round

        #Step 1: Standard FedAvg aggregation
        aggregated_parameters, metrics = super().aggregate_fit(server_round, results, failures)

        #Characterize clients through clustering
        if server_round in WLRC_ROUNDS:
            print("Clustering client weights...")

            # Extract weights and flatten
            weights = []
            client_ids = []
            IIDness_of_clients = []

            #Extract clients information
            for client_proxy, fit_res in results:
                ndarrays = parameters_to_ndarrays(fit_res.parameters)
                last_layer = ndarrays[-1]
                flat_weights = last_layer.flatten()
                weights.append(flat_weights)
                client_ids.append(fit_res.metrics['id'])
                IIDness_of_clients.append(fit_res.metrics['IIDness'])

            weights = np.array(weights)
            weights = normalize(weights, norm='l2')

            #Perform clustering and project weights
            kmeans = KMeans(n_clusters=self.num_clusters, random_state=SEED).fit(weights)
            klabels = kmeans.labels_
            #populate client_clusters bookeeping
            for cid, label in zip(client_ids, klabels):
                self.client_clusters[cid] = int(label)

            #Focus on characterization of least nonIID clusters
            IID_clusters = set()
            for cid, iidness in zip(client_ids, IIDness_of_clients):
                if iidness:
                    IID_clusters.add(self.client_clusters[cid])
            print(f"Cluster(s) where IID clients are present are: {IID_clusters}")

            clusters_characterization = {}
            for cluster in IID_clusters:
                clusters_characterization[cluster] = {"IID": 0, "nonIID": 0}
                for cid, iidness in zip(client_ids, IIDness_of_clients):
                    if self.client_clusters[cid] == cluster and iidness:
                        clusters_characterization[cluster]["IID"] += 1
                    elif self.client_clusters[cid] == cluster and not iidness:
                        clusters_characterization[cluster]["nonIID"] += 1

            print(f"Clusters with IID clients are characterized like this: {clusters_characterization}")

            #Plot clusters
            print("Projecting with t-SNE...")
            tsne = TSNE(n_components=2, perplexity=5, random_state=SEED)
            weights_2d = tsne.fit_transform(weights)

            plt.figure(figsize=(8, 6))
            for cluster_id in range(self.num_clusters):
                idxs = [i for i, label in enumerate(klabels) if label == cluster_id]
                xs = [weights_2d[i][0] for i in idxs]
                ys = [weights_2d[i][1] for i in idxs]
                plt.scatter(xs, ys, label=f"Cluster {cluster_id}", s=50)

            for i, cid in enumerate(client_ids):
                plt.annotate(cid, (weights_2d[i][0], weights_2d[i][1]), fontsize=8, alpha=0.6)

            plt.title("Client Weights Projected via t-SNE")
            plt.legend()
            plt.grid(True)
            plt.tight_layout()
            plt.savefig(f"{LOG_DIR}/tsne_clusters_round_{server_round}_{self.mode}.png")
            plt.close()

            with open(f"{LOG_DIR}/{self.mode}_clusters.txt", 'a') as f:
              f.write(f"Round: {server_round}\n")
              f.write(f"Cluster(s) where IID clients are present are: {IID_clusters}\n")
              f.write(f"Clusters with IID clients are characterized like this: {clusters_characterization}\n")
              f.write(f"Cluster assignments per client are: \n {self.client_clusters}\n\n")
              f.close()



            #Activate WLRC model choice
            if self.mode == "WLRC":
                #Extract all estimated complexities and aggregate them by cluster
                R = {}
                for _, fit_res in results:
                    R[fit_res.metrics['id']] = fit_res.metrics['R_score']
                print(f"Rademacher complexities are {R} \n for cluster assignments {self.client_clusters}")

                r_score, ranking = aggregate_and_rank_score_per_cluster(R, self.client_clusters)
                print(f"Rademacher scores per cluster are {r_score} and ranking of clusters is {ranking}.")

                self.top_clusters = ranking[0:TOP_K]

            elif self.mode == "LRC_Sampling":
                R = {}
                for _, fit_res in results:
                    R[fit_res.metrics['id']] = fit_res.metrics['R_score']
                print(f"Rademacher complexities are {R} \n for cluster assignments {self.client_clusters}")

                node_choice = get_top_k_nodes_per_cluster(1, R, self.client_clusters)

                print(f"Chosen nodes for training are {node_choice}.")

                self.node_choice = node_choice

            elif self.mode == "Mixed_Sampling":
                R = {}
                #Mix LRC and WLRC Sampling
                for _, fit_res in results:
                    R[fit_res.metrics['id']] = fit_res.metrics['R_score']
                print(f"Rademacher complexities are {R} \n for cluster assignments {self.client_clusters}")

                node_choice = get_top_k_nodes_per_cluster(1, R, self.client_clusters)
                r_score, ranking = aggregate_and_rank_score_per_cluster(R, self.client_clusters)
                self.top_clusters = ranking[0:TOP_K]

                #Add best cluster nodes to old node_choice
                for cluster in self.top_clusters:
                  #Find nodes of best cluster
                  nodes = {}
                  for (k, v) in self.client_clusters.items():
                    nodes[v] = nodes.get(v, []) + [k]

                  #best_cluster_nodes = nodes[cluster]
                  #Update node choice
                  node_choice.update(nodes[cluster])

                print(f"Chosen nodes for training are {node_choice}.")
                self.node_choice = node_choice





        return aggregated_parameters, metrics

    #redefined for logging purposes
    def aggregate_evaluate(self, round, results, failures):
        aggregated_result = super().aggregate_evaluate(round, results, failures)

        if round == 1 and self.mode == 'randselFedavg':
            with open(f"{LOG_DIR}/summary.txt", "a") as f:
                f.write(f"RUN {run_id}\n")

        if aggregated_result is not None:
            loss, metrics = aggregated_result
            with open(f"{LOG_DIR}/summary.txt", "a") as f:
                f.write(f"Round {round}: loss={loss:.4f}, metrics={metrics}\n")

        return aggregated_result


    #Configure fit redefined to filter clients in WLRC setting based on cluster scores
    def configure_fit(self, server_round, parameters, client_manager):
        config = self.on_fit_config_fn(server_round)  # Additional config if needed

        #Get available clients
        available_clients = list(client_manager.all().values()) #client_manager.all() contain 'flower_cid' : clientproxy mappings

        #Map ids that change at each round of flower to stable ids of FedArtML that are sequential
        flowerids2cid.clear()
        for client in available_clients:
            flowerids2cid[int(client.cid)] = len(flowerids2cid) + 1

        if self.mode == "WLRC" and hasattr(self, "top_clusters"):

            # Filter clients in the top-ranked clusters
            filtered_clients = [
                c for c in available_clients
                if self.client_clusters[flowerids2cid[int(c.cid)]] in self.top_clusters
            ]

        elif (self.mode == "LRC_Sampling" or self.mode == 'Mixed_Sampling') and hasattr(self, "node_choice"):

          # Filter clients to find the chosen clients among the clusters
          filtered_clients = [
              c for c in available_clients
              if flowerids2cid[int(c.cid)] in self.node_choice
          ]

        else:
            filtered_clients = available_clients

        #Sorted for reproducibility
        filtered_clients = sorted(filtered_clients, key=lambda c: flowerids2cid[int(c.cid)])



        # Sample from the filtered clients
        num_clients = max(1, int(self.fraction_fit * len(filtered_clients)))
        sampled_clients = random.sample(filtered_clients, num_clients)

        return [(client, FitIns(parameters=parameters, config=config)) for client in sampled_clients]



## Flower Client

In [ ]:
# Flower client
class FedArtFlowerClient(fl.client.NumPyClient):
    def __init__(self, x, y, id, IIDness, mode):
        self.model = Net()
        self.device = DEVICE
        self.model.to(self.device)
        self.id = id
        self.mode = mode
        assert self.mode in PERMITTED_MODES, "Mode must be in PERMITTED MODES"

        x_train, x_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=SEED, #stratify=y  # optional stratify
        )
        x_train = np.array(x_train)
        x_test = np.array(x_test)

        x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)


        self.train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=32, shuffle=True, generator=generator)
        self.test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.isIID = True if IIDness == "with_class_completion" else False

    def get_parameters(self, config=None):
        return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    # training function for the clients
    def fit(self, parameters, config=None):
        self.set_parameters(parameters)
        self.model.train()
        R = 0 #Rademacher complexity estimation

        for epoch in range(config["local_training_epochs"]):
            for batch_x, batch_y in self.train_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(batch_x)
                loss = self.criterion(output, batch_y)
                loss.backward()
                self.optimizer.step()

                #Rademacher complexity is estimated only at the last epoch to assess effects of nonIIDness on the training
                if epoch == config["local_training_epochs"] - 1 and config["server_round"] in WLRC_ROUNDS and (self.mode == 'WLRC' or self.mode == 'LRC_Sampling' or self.mode == 'Mixed_Sampling'):
                    #Update rademacher complexity estimation
                    R += estimate_Rk(Q, R, len(batch_x), C, self.model, batch_x)

        R = R / len(self.train_loader)

        return self.get_parameters(), len(self.train_loader.dataset), {'id' : self.id,
                                                                       'IIDness': self.isIID,
                                                                       'R_score': R.item() if config["server_round"] in WLRC_ROUNDS and (self.mode == 'WLRC' or self.mode == 'LRC_Sampling' or self.mode == 'Mixed_Sampling') else 0,
                                                                       }

    # evaluation function for the clients
    def evaluate(self, parameters, config=None):
        self.set_parameters(parameters)
        self.model.eval()

        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch_x, batch_y in self.test_loader:
                batch_x, batch_y = batch_x.to(self.device), batch_y.to(self.device)
                output = self.model(batch_x)

                loss = self.criterion(output, batch_y)
                total_loss += loss.item() * batch_x.size(0)  # sum up batch loss

                preds = output.argmax(dim=1)
                correct += (preds == batch_y).sum().item()
                total += batch_y.size(0)

        avg_loss = total_loss / total
        accuracy = correct / total
        return avg_loss, total, {"id": self.id, "accuracy": accuracy}

#CLIENT IDS are the SAME to the FedArtML IDS
#Function defining in the simulation data distribution of client cid
def client_fn(context: Context) -> fl.client.NumPyClient:

    id = flowerids2cid[int(context.node_id)]
    client_id = f"Local_node_{id}"

    #Check IIDness of client
    if is_iid(id, clients_glob_dic):
      IIDness = "with_class_completion"
    else:
      IIDness = "without_class_completion"

    client_data = clients_glob_dic[IIDness][client_id]
    x = [item[0] for item in client_data]  # list of x_i numpy arrays
    y = [item[1] for item in client_data]  # list of labels

    log = f"[Client {id}] called with {len(x)} samples and is {IIDness}"

    if IIDness == "without_class_completion":
      miss = []
      for i in range(0,10):
        if i not in y:
          miss.append(i)

    log = log + '.' if IIDness == "with_class_completion" else log + f" and misses labels {miss}."
    print(log)

    return FedArtFlowerClient(x, y, id, IIDness, MODE).to_client()

## Flower Server

In [ ]:
#Server calls this whenever it receives evaluate metrics from the clients
def evaluate_metrics_aggregation_fn(metrics_list):
    if not metrics_list:
        return {}
    print(f"Metrics are: {metrics_list}")
    total_examples = sum(num_examples for num_examples, _ in metrics_list)
    weighted_accuracy = sum(
        num_examples * metrics["accuracy"] for num_examples, metrics in metrics_list
    ) / total_examples

    return {"accuracy": weighted_accuracy}

#Server calls this whenever it receives fit metrics from the clients
#def fit_metrics_aggregation_fn(metrics_list):
 #  print(metrics_list)
 #  return {}

#Server calls this to decide what to send as "config" dict to the fit function of the client
def on_fit_config_fn(server_round: int):
  config = {
        "server_round": server_round,  #current round of federated learning
        "local_training_epochs": T,
    }
  return config

In [ ]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    #Initialize parameters for reproducibility
    params = [val.cpu().numpy() for _, val in Net().state_dict().items()]

    strategy = CustomFedAvg(
        fraction_fit=fraction_fit,   # fraction of clients used for training each round
        fraction_evaluate=1,  # fraction of clients used for evaluation
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=NUM_CLIENTS,
        evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
        initial_parameters=ndarrays_to_parameters(params),
        on_fit_config_fn=on_fit_config_fn,
        num_cluster=NUM_CLUSTERS,
        mode=MODE,
    )

    # Configure the server for NUM_ROUNDS of training
    config = ServerConfig(num_rounds=NUM_ROUNDS)

    return ServerAppComponents(strategy=strategy, config=config)

#Start simulation

In [ ]:
#RUN SIMULATION
assert MODE != 'randselFedavg', "For this mode run the cell below"
backend_config = {"client_resources": None}
if DEVICE.type == "cuda":
    backend_config = {"client_resources": {"num_gpus": 1}}

#Construct the ClientApp passing the client generation function
client_app = ClientApp(client_fn=client_fn)

#Create your ServerApp passing the server generation function
server_app = ServerApp(server_fn=server_fn)
print("MODE IS ", MODE)
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=20, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 100 clients (out of 100)


MODE IS  WLRC


(pid=2047) 2025-06-16 07:16:41.258421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2047) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=2047) E0000 00:00:1750058201.674020    2047 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2047) E0000 00:00:1750058201.723823    2047 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(ClientAppActor pid=2047) [Client 24] called with 215 samples and is without_class_completion and misses labels [2, 3, 5, 6, 7, 8, 9].
(ClientAppActor pid=2047) [Client 63] called with 1701 samples and is without_class_completion and misses labels [3, 5, 6, 7, 8, 9].
(ClientAppActor pid=2047) [Client 31] called with 748 samples and is without_class_completion and misses labels [3, 5, 6, 8, 9].
(ClientAppActor pid=2047) [Client 64] called with 1732 samples and is without_class_completion and misses labels [5, 9].
(ClientAppActor pid=2047) [Client 81] called with 318 samples and is without_class_completion and misses labels [1, 2, 4, 5, 7, 9].
(ClientAppActor pid=2047) [Client 26] called with 829 samples and is without_class_completion and misses labels [0, 1, 3, 9].
(ClientAppActor pid=2047) [Client 62] called with 27 samples and is without_class_completion and misses labels [0, 1, 2, 6, 9].
(ClientAppActor pid=2047) [Client 38] called with 644 samples and is without_class_completion an

INFO :      aggregate_fit: received 100 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


...

Metrics are: [(53, {'id': 73, 'accuracy': 0.8867924528301887}), (76, {'id': 58, 'accuracy': 0.3026315789473684}), (38, {'id': 8, 'accuracy': 0.07894736842105263}), (43, {'id': 86, 'accuracy': 0.9302325581395349}), (19, {'id': 80, 'accuracy': 0.47368421052631576}), (101, {'id': 60, 'accuracy': 0.8613861386138614}), (289, {'id': 9, 'accuracy': 0.0}), (228, {'id': 20, 'accuracy': 0.0043859649122807015}), (237, {'id': 6, 'accuracy': 0.1518987341772152}), (166, {'id': 26, 'accuracy': 0.05421686746987952}), (131, {'id': 43, 'accuracy': 0.0}), (35, {'id': 13, 'accuracy': 0.4}), (27, {'id': 90, 'accuracy': 0.8518518518518519}), (175, {'id': 39, 'accuracy': 0.03428571428571429}), (386, {'id': 23, 'accuracy': 0.0025906735751295338}), (252, {'id': 16, 'accuracy': 0.46825396825396826}), (117, {'id': 32, 'accuracy': 0.2564102564102564}), (132, {'id': 53, 'accuracy': 0.03787878787878788}), (8, {'id': 82, 'accuracy': 0.125}), (320, {'id': 59, 'accuracy': 0.0875}), (113, {'id': 28, 'accuracy': 0.07964

INFO :      aggregate_fit: received 12 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


(ClientAppActor pid=2047) [Client 59] called with 1596 samples and is without_class_completion and misses labels [1, 2, 3, 5, 6, 7, 8, 9].
(ClientAppActor pid=2047) [Client 29] called with 252 samples and is without_class_completion and misses labels [3, 4, 5, 6, 9].
(ClientAppActor pid=2047) [Client 35] called with 740 samples and is without_class_completion and misses labels [1, 2, 4, 5, 6, 7, 8, 9].
(ClientAppActor pid=2047) [Client 56] called with 508 samples and is without_class_completion and misses labels [1, 2, 4, 6, 7, 9].
(ClientAppActor pid=2047) [Client 53] called with 659 samples and is without_class_completion and misses labels [1, 2, 3, 7, 8].
(ClientAppActor pid=2047) [Client 72] called with 554 samples and is without_class_completion and misses labels [4, 5, 7, 8, 9].
(ClientAppActor pid=2047) [Client 90] called with 131 samples and is without_class_completion and misses labels [1, 3, 4, 6, 7, 8, 9].
(ClientAppActor pid=2047) [Client 41] called with 1020 samples and is

INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 20 round(s) in 5699.55s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.685444570392734
INFO :      		round 2: 1.3013265351142507
INFO :      		round 3: 1.1418035262308328
INFO :      		round 4: 1.0636914908238926
INFO :      		round 5: 1.0204085217827805
INFO :      		round 6: 0.9895497813468237
INFO :      		round 7: 0.9632678048504019
INFO :      		round 8: 0.9367444566231314
INFO :      		round 9: 0.9082127783212874
INFO :      		round 10: 0.8762997839970512
INFO :      		round 11: 3.5117888994097552
INFO :      		round 12: 6.656887989575508
INFO :      		round 13: 9.306855974190688
INFO :      		round 14: 11.331919735140868
INFO :      		round 15: 12.59405068361321
INFO :      		round 16: 13.292375288829785
INFO :      		round 17: 13.664698775972314
INFO :      		round 18: 13.910416391112372
INFO :      		round 19: 14.033394407933296
IN

(ClientAppActor pid=2047) [Client 15] called with 584 samples and is without_class_completion and misses labels [1, 2, 3].
Metrics are: [(67, {'id': 91, 'accuracy': 0.05970149253731343}), (386, {'id': 23, 'accuracy': 0.0025906735751295338}), (117, {'id': 32, 'accuracy': 0.2564102564102564}), (42, {'id': 50, 'accuracy': 0.09523809523809523}), (121, {'id': 93, 'accuracy': 0.8429752066115702}), (251, {'id': 99, 'accuracy': 0.16733067729083664}), (38, {'id': 8, 'accuracy': 0.07894736842105263}), (67, {'id': 18, 'accuracy': 0.373134328358209}), (98, {'id': 44, 'accuracy': 0.0}), (129, {'id': 38, 'accuracy': 0.1937984496124031}), (166, {'id': 26, 'accuracy': 0.05421686746987952}), (248, {'id': 96, 'accuracy': 0.09274193548387097}), (43, {'id': 86, 'accuracy': 0.9302325581395349}), (51, {'id': 29, 'accuracy': 0.7450980392156863}), (14, {'id': 7, 'accuracy': 0.7142857142857143}), (149, {'id': 75, 'accuracy': 0.2348993288590604}), (128, {'id': 65, 'accuracy': 0.2265625}), (89, {'id': 77, 'accur

# Start RandselFedAvg simulation (more runs)

In [ ]:
import re
from collections import defaultdict

def parse_federated_results(filepath):
    round_data = defaultdict(lambda: {'loss': [], 'accuracy': []})
    round_pattern = re.compile(r"Round (\d+): loss=([\d.]+), metrics={'accuracy': ([\d.]+)}")

    with open(filepath, 'r') as file:
        for line in file:
            match = round_pattern.search(line)
            if match:
                round_num = int(match.group(1))
                loss = float(match.group(2))
                accuracy = float(match.group(3))
                round_data[round_num]['loss'].append(loss)
                round_data[round_num]['accuracy'].append(accuracy)

    # Compute averages
    averaged_results = {}
    for rnd, values in round_data.items():
        avg_loss = sum(values['loss']) / len(values['loss'])
        avg_accuracy = sum(values['accuracy']) / len(values['accuracy'])
        averaged_results[rnd] = {
            'avg_loss': avg_loss,
            'avg_accuracy': avg_accuracy
        }

    return averaged_results

def get_remaining(file_path, total_runs=10):
    run_pattern = re.compile(r"RUN (\d+)")
    found_runs = set()

    with open(file_path, 'r') as f:
        for line in f:
            match = run_pattern.match(line.strip())
            if match:
                run_id = int(match.group(1))
                found_runs.add(run_id)

    max_run = max(found_runs) if found_runs else 0
    if max_run == 10:
      return None, None
    missing_runs = sorted(set(range(1, total_runs + 1)) - found_runs)
    runs_left = total_runs - len(found_runs)
    restart_from = missing_runs[0] if missing_runs else max_run + 1

    print(f"Max run number reached: {max_run}")
    print(f"Runs left to reach {total_runs}: {runs_left}")
    print(f"Restart from run: {restart_from}")

    return runs_left, restart_from

In [ ]:
#RUN randselFedavg experiments to get empirical expected results of the runs
N_RUNS = 10
#Restart training from wher it was left if colab interrupted it
_, restart_id = get_remaining(f"{LOG_DIR}/summary.txt")
restart_id = 1 if restart_id is None else restart_id

for run_id in range(restart_id, N_RUNS + 1):
    print(f"\n--- Starting run {run_id}/{N_RUNS} ---")

    random.seed(run_id)
    np.random.seed(run_id)
    torch.manual_seed(run_id)

    backend_config = {"client_resources": None}
    if DEVICE.type == "cuda":
        backend_config = {"client_resources": {"num_gpus": 1}}

    client_app = ClientApp(client_fn=client_fn)
    server_app = ServerApp(server_fn=server_fn)

    run_simulation(
        server_app=server_app,
        client_app=client_app,
        num_supernodes=NUM_CLIENTS,
        backend_config=backend_config,
    )

final_results = parse_federated_results(f"{LOG_DIR}/summary.txt")

with open(f"{LOG_DIR}/final_results.txt", "w") as f:
    for rnd, values in final_results.items():
        f.write(f"Round {rnd}: avg_loss={values['avg_loss']:.4f}, avg_accuracy={values['avg_accuracy']:.4f}\n")


DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=20, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)


Max run number reached: 5
Runs left to reach 10: 5
Restart from run: 6

--- Starting run 6/10 ---


(pid=3719) 2025-06-16 07:36:43.144230: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=3719) WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
(pid=3719) E0000 00:00:1750059403.215014    3719 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=3719) E0000 00:00:1750059403.234423    3719 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(ClientAppActor pid=3719) [Client 34] called with 17 samples and is without_class_completion and misses labels [0, 1, 5, 7, 8].
(ClientAppActor pid=3719) [Client 85] called with 274 samples and is without_class_completion and misses labels [0, 1, 3, 4, 8].
(ClientAppActor pid=3719) [Client 5] called with 687 samples and is without_class_completion and misses labels [0, 2, 3, 5, 6, 7, 8, 9].
(ClientAppActor pid=3719) [Client 98] called with 647 samples and is without_class_completion and misses labels [1, 3, 4, 6, 7, 8, 9].
(ClientAppActor pid=3719) [Client 1] called with 1786 samples and is without_class_completion and misses labels [1, 2, 3, 4, 5, 6, 7, 8, 9].
(ClientAppActor pid=3719) [Client 19] called with 161 samples and is without_class_completion and misses labels [0, 2, 3, 5, 6, 7].
(ClientAppActor pid=3719) [Client 76] called with 1488 samples and is without_class_completion and misses labels [0, 3, 4, 5, 6, 7, 8, 9].
(ClientAppActor pid=3719) [Client 61] called with 124 sampl

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


...

Metrics are: [(15, {'id': 46, 'accuracy': 0.8666666666666667}), (14, {'id': 70, 'accuracy': 0.9285714285714286}), (204, {'id': 41, 'accuracy': 0.8333333333333334}), (35, {'id': 13, 'accuracy': 0.8571428571428571}), (76, {'id': 58, 'accuracy': 0.868421052631579}), (33, {'id': 19, 'accuracy': 0.8787878787878788}), (67, {'id': 18, 'accuracy': 0.835820895522388}), (25, {'id': 61, 'accuracy': 0.2}), (228, {'id': 20, 'accuracy': 0.8421052631578947}), (3, {'id': 51, 'accuracy': 0.3333333333333333}), (289, {'id': 47, 'accuracy': 0.9204152249134948}), (43, {'id': 86, 'accuracy': 0.813953488372093}), (180, {'id': 69, 'accuracy': 0.9333333333333333}), (93, {'id': 2, 'accuracy': 0.9247311827956989}), (203, {'id': 83, 'accuracy': 0.8768472906403941}), (8, {'id': 82, 'accuracy': 0.5}), (101, {'id': 60, 'accuracy': 0.9405940594059405}), (121, {'id': 78, 'accuracy': 0.8264462809917356}), (188, {'id': 79, 'accuracy': 0.9840425531914894}), (13, {'id': 89, 'accuracy': 1.0}), (138, {'id': 5, 'accuracy': 0

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 100 clients (out of 100)


(ClientAppActor pid=72559) [Client 77] called with 444 samples and is without_class_completion and misses labels [0, 1, 4, 6, 7, 9].
(ClientAppActor pid=72559) [Client 67] called with 83 samples and is without_class_completion and misses labels [1, 3, 6, 7].
(ClientAppActor pid=72559) [Client 49] called with 314 samples and is without_class_completion and misses labels [1, 7, 9].
(ClientAppActor pid=72559) [Client 44] called with 486 samples and is without_class_completion and misses labels [2, 3, 8].
(ClientAppActor pid=72559) [Client 58] called with 380 samples and is without_class_completion and misses labels [0, 3, 5, 8].
(ClientAppActor pid=72559) [Client 12] called with 240 samples and is without_class_completion and misses labels [0, 1, 2, 4, 6, 8, 9].
(ClientAppActor pid=72559) [Client 100] called with 41 samples and is with_class_completion.
(ClientAppActor pid=72559) [Client 50] called with 206 samples and is without_class_completion and misses labels [0, 1, 4, 5, 7, 8].
(Cli

INFO :      aggregate_evaluate: received 100 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 20 round(s) in 4115.46s
INFO :      	History (loss, distributed):
INFO :      		round 1: 3.4011900675148006
INFO :      		round 2: 4.875386632216018
INFO :      		round 3: 3.206060618620119
INFO :      		round 4: 3.309479112193435
INFO :      		round 5: 3.7047311360096873
INFO :      		round 6: 3.1101622600923746
INFO :      		round 7: 1.0679305157269805
INFO :      		round 8: 0.9530382813934235
INFO :      		round 9: 0.9135988860076462
INFO :      		round 10: 1.0595085569367098
INFO :      		round 11: 0.781716353915113
INFO :      		round 12: 0.4899277547289273
INFO :      		round 13: 0.7417539714752124
INFO :      		round 14: 0.9636789162677378
INFO :      		round 15: 0.5019086625244866
INFO :      		round 16: 0.5995263581549201
INFO :      		round 17: 0.44952185253730004
INFO :      		round 18: 0.4834044034052137
INFO :      		round 19: 0.6525925173448766
I

(ClientAppActor pid=72559) [Client 62] called with 27 samples and is without_class_completion and misses labels [0, 1, 2, 6, 9].
Metrics are: [(8, {'id': 82, 'accuracy': 0.75}), (347, {'id': 64, 'accuracy': 0.0345821325648415}), (42, {'id': 50, 'accuracy': 0.7619047619047619}), (9, {'id': 100, 'accuracy': 0.8888888888888888}), (243, {'id': 97, 'accuracy': 0.9835390946502057}), (141, {'id': 42, 'accuracy': 0.9361702127659575}), (197, {'id': 27, 'accuracy': 0.3350253807106599}), (27, {'id': 55, 'accuracy': 0.8518518518518519}), (248, {'id': 96, 'accuracy': 0.24193548387096775}), (17, {'id': 67, 'accuracy': 0.5294117647058824}), (47, {'id': 11, 'accuracy': 0.2765957446808511}), (252, {'id': 16, 'accuracy': 0.8214285714285714}), (113, {'id': 28, 'accuracy': 0.1504424778761062}), (320, {'id': 59, 'accuracy': 0.75625}), (98, {'id': 44, 'accuracy': 0.8469387755102041}), (132, {'id': 53, 'accuracy': 0.5984848484848485}), (289, {'id': 9, 'accuracy': 0.8546712802768166}), (166, {'id': 26, 'accur